In [2]:
import os      
os.chdir(r"/groups/umcg-wijmenga/tmp01/users/umcg-nribeiro/.conda/pkgs") 

In [3]:
os.environ['R_HOME'] = '/apps/software/R/4.2.2-foss-2022a-bare/lib64/R'

In [ ]:
pip install sccoda

In [2]:
pip install tensorflow==2.11.*

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Setup
import importlib
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt

from sccoda.util import comp_ana as mod
from sccoda.util import cell_composition_data as dat
from sccoda.util import data_visualization as viz

import sccoda.datasets as scd

In [ ]:
out_path = '/groups/umcg-wijmenga/tmp01/users/umcg-aramirezsanchez/umcg-nribeiro/NR03_scRNAseq/ongoing/outputs/cell_proportion_analysis/'

In [4]:
# Load data
cell_counts = pd.read_csv('/groups/umcg-wijmenga/tmp01/users/umcg-aramirezsanchez/umcg-nribeiro/NR03_scRNAseq/ongoing/outputs/subclustering/epithelial_cells_counts.csv')
cell_counts

,sample,age,sex,batch,status,EEC,EEC A/M,EEC D,EEC I/L,EEC K,...,Enterocytes mature (zone 4),Enterocytes mature (zone 5),Goblet cells,Paneth cells,Stem cells 1,Stem cells 2,TA cells 1,TA cells 2,Tuft cells,BEST4 cells
0,CeD1,10,M,A,CeD,4,4,21,9,8,...,41,1,11,42,175,31,61,67,28,0
1,CeD2,14,F,A,CeD,0,8,22,8,9,...,25,25,13,5,164,67,48,39,54,7
2,CeD2,14,F,F,CeD,1,2,4,2,3,...,15,18,9,0,163,30,58,24,7,3
3,CeD3,14,F,A,CeD,1,25,3,14,3,...,226,95,21,7,212,130,73,56,67,27
4,CeD4,17,F,A,CeD,0,4,2,2,0,...,21,1,19,35,74,34,48,20,9,0
5,CeD4,17,F,F,CeD,2,5,3,5,4,...,18,0,20,7,31,13,41,14,9,1
6,Ctrl1,15,F,A,Ctrl,20,15,10,11,30,...,52,126,26,51,105,105,29,17,96,16
7,Ctrl1,15,F,F,Ctrl,20,17,12,10,4,...,27,38,25,19,177,81,39,31,55,10
8,Ctrl2,11,M,F,Ctrl,14,10,17,10,11,...,162,270,15,12,149,175,91,35,64,9
9,Ctrl4,12,M,A,Ctrl,0,4,2,14,0,...,133,291,30,8,96,99,20,13,42,24


In [5]:
# Convertind to ann data
data_all = dat.from_pandas(cell_counts, covariate_columns = ['sample', 'age', 'sex', 'batch', 'status'])
print(data_all.obs)

   sample  age sex batch status
0    CeD1   10   M     A    CeD
1    CeD2   14   F     A    CeD
2    CeD2   14   F     F    CeD
3    CeD3   14   F     A    CeD
4    CeD4   17   F     A    CeD
5    CeD4   17   F     F    CeD
6   Ctrl1   15   F     A   Ctrl
7   Ctrl1   15   F     F   Ctrl
8   Ctrl2   11   M     F   Ctrl
9   Ctrl4   12   M     A   Ctrl
10  Ctrl5   15   F     A   Ctrl


In [ ]:
# Building the model and running, HMC chain length = 400,000, burn-in = 10,000, FDR = 0.05
model = mod.CompositionalAnalysis(data_all, formula = "~C(status, Treatment('Ctrl')) + age + sex + batch", reference_cell_type = "automatic")
results = model.sample_hmc(num_results = 200000, num_burnin = 10000)
results.summary()

Automatic reference selection! Reference cell type set to EEC I/L
Zero counts encountered in data! Added a pseudocount of 0.5.


2023-02-23 10:21:58.873525: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-23 10:21:58.873603: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-23 10:21:58.873641: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (gs-vcompute05): /proc/driver/nvidia/version does not exist
2023-02-23 10:21:58.874559: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))


2023-02-23 10:22:03.890082: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7fce2400fc70 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-02-23 10:22:03.890241: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Host, Default Version
  0%|                                                                                        | 0/200000 [00:00<?, ?it/s]2023-02-23 10:22:03.972905: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-02-23 10:22:04.950918: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
 63%|██████████████████████████████████████████████▎                           | 125175/200000 [18:17<11:10, 111.63it/s]

In [ ]:
# Saving the results (effects)
pd.DataFrame.to_csv(results.effect_df, 'sCODA_epithelia_results_effect_summary.csv')